In [2]:
import pandas as pd
import numpy as np
import sqlite3
import os

import sys
sys.path.append("../../code/data_processing/")
import taxonomy

Python-dotenv could not parse statement starting at line 4


INFO: Pandarallel will run on 5 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# parse WFO

In [3]:
version = "v.1.12"
name_resolution_dir = f"../../data/name_resolution/"
db_dir = f"{name_resolution_dir}/"
if not os.path.exists(db_dir):
    if not os.path.exists(f"{name_resolution_dir}WFOTaxonomicBackbone_v.1.12_20221221.zip"):
        res = os.system(f"cd {name_resolution_dir};wget http://104.198.143.165/files/WFO_Backbone/_WFOCompleteBackbone/archive/WFOTaxonomicBackbone_v.1.12_20221221.zip")
if len(os.listdir(db_dir)) == 0:
    res = os.system(f"unzip {name_resolution_dir}WFOTaxonomicBackbone_v.1.12_20221221 {db_dir}")

zip_paths = [p for p in os.listdir(db_dir) if p.endswith('.zip') or p.endswith(".gz")]
input_path = f"{db_dir}classification.txt" #if version != "current" else f"{db_dir}030_name_matching_big.csv"
db_path = f"{name_resolution_dir}wfo.db"

required_cols = ["Specie", "Author", "Accepted_Specie", "Accepted_author", "Status"]
table_names = ["Accepted_Names", "Synonym_Names", "Unresolved_Names"]

In [12]:
for zip_path in zip_paths:
    res = os.system(f"cd {db_dir}; {'unzip' if zip_path.endswith('zip') else 'gzip -d'} {zip_path}")

In [13]:
classification = pd.read_csv(input_path, sep="\t" if version != "current" else ",", on_bad_lines="skip", low_memory=False, encoding = "ISO-8859-1")
taxon_id_col = [c for c in classification.columns if c.endswith("taxonID")][0]
classification.rename(columns={"scientificName": "Specie",
                                "scientificNameAuthorship" :"Author",
                                "taxonomicStatus": "Status",
                                taxon_id_col: "taxonID"}, inplace=True)

classification.replace({"accepted": "ACCEPTED", "synonym": "SYNONYM", "unplaced": "UNRESOLVED"}, inplace=True)

if "acceptedNameUsageID" in classification.columns:
    id_to_name = classification.set_index("taxonID")["Specie"].to_dict()
    id_to_author = classification.set_index("taxonID")["Author"].to_dict()
    classification["Accepted_Specie"] = classification.apply(lambda record: id_to_name.get(record.acceptedNameUsageID, np.nan) if pd.notna(record.acceptedNameUsageID) else record.Specie, axis=1)
    classification["Accepted_author"] = classification.apply(lambda record: id_to_author.get(record.acceptedNameUsageID, np.nan) if pd.notna(record.acceptedNameUsageID) else record.Author, axis=1)      

classification["New_status"] = classification[["Specie", "Accepted_Specie", "Status"]].apply(lambda record: "ACCEPTED" if record.Specie == record.Accepted_Specie else record.Status, axis=1)

In [14]:
classification["New_status"] = classification[["Specie", "Accepted_Specie", "Status"]].apply(lambda record: "ACCEPTED" if record.Specie == record.Accepted_Specie else record.Status, axis=1)

curate based on two known discrepancies with the webserver

In [15]:
test_1_success = "wfo-0000154982" not in classification.taxonID.tolist()
test_2_success = classification.loc[classification.taxonID == "wfo-0000409033", "Status"].values[0] == "SYNONYM"
inconsistent_db = not test_1_success or not test_2_success
if inconsistent_db:
    print("warning! db exhibits inconsistencies with the web-server")

write database

In [16]:
accepted = classification.loc[classification.Status == "ACCEPTED"]
synonyms = classification.loc[classification.Status == "SYNONYM"]
unres = classification.loc[~classification.Status.isin(["ACCEPTED", "SYNONYM"])]
print(f"# accepted records = {accepted.shape[0]:,}\n# synonym records = {synonyms.shape[0]:,}\n# unresolved records = {unres.shape[0]:,}")

# accepted records = 444,571
# synonym records = 827,578
# unresolved records = 152,912


In [ ]:
if not os.path.exists(db_path):
    con = sqlite3.connect(db_path)
    accepted.to_sql(name="Accepted_Names", con=con)
    synonyms.to_sql(name="Synonym_Names", con=con)
    unres.to_sql(name="Unresolved_Names", con=con)
    con.close()

# parse ITIS

In [79]:
itis_db_path = "../../data/name_resolution/itisSqlite042623/ITIS.sqlite"
itis_processed_db_path = "../../data/name_resolution/itis.db"
connection = sqlite3.connect(itis_db_path)

In [27]:
cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('HierarchyToRank',), ('change_comments',), ('change_operations',), ('change_tracks',), ('chg_operation_lkp',), ('comments',), ('experts',), ('geographic_div',), ('hierarchy',), ('jurisdiction',), ('kingdoms',), ('longnames',), ('nodc_ids',), ('other_sources',), ('publications',), ('reference_links',), ('reviews',), ('strippedauthor',), ('synonym_links',), ('taxon_authors_lkp',), ('taxon_unit_types',), ('taxonomic_units',), ('tu_comments_links',), ('vern_ref_links',), ('vernaculars',)]


In [18]:
# relevant polllinator kingdoms: Insecta,  Metazoa 
kingdoms_query = (f"SELECT * FROM kingdoms")
kingdoms_data = pd.read_sql_query(kingdoms_query, connection)
relevant_kingdom_id = kingdoms_data.query("kingdom_name == 'Animalia'").kingdom_id.values[0]
relevant_kingdom_id

5

In [20]:
taxonomic_units_query = f"SELECT * FROM taxonomic_units WHERE kingdom_id IS {relevant_kingdom_id}"
taxonomic_units_data = pd.read_sql_query(taxonomic_units_query, connection)
taxonomic_units_data.complete_name = taxonomic_units_data.complete_name.str.capitalize()

In [35]:
tax_author_q = f"SELECT * FROM taxon_authors_lkp WHERE  kingdom_id IS {relevant_kingdom_id}"
tax_authors_data = pd.read_sql_query(tax_author_q, connection)
taxonomic_units_data = taxonomic_units_data.merge(tax_authors_data, on="taxon_author_id", how="left")

In [59]:
rank_data_q = f"SELECT * FROM taxon_unit_types"
rank_data = pd.read_sql_query(rank_data_q, connection)
taxonomic_units_data = taxonomic_units_data.merge(rank_data, on="rank_id", how="left")

In [100]:
taxonomic_units_data.to_csv("../../data/name_resolution/itis_taxonomic_data.csv")

In [94]:
accepted_names = taxonomic_units_data.query("unaccept_reason.isna()")#[["complete_name", "taxon_author", "
synonym_names = taxonomic_units_data.loc[taxonomic_units_data.unaccept_reason.isin(['junior synonym', 'junior homonym', 'unnecessary replacement'])]
unresolved_names = taxonomic_units_data.loc[(~taxonomic_units_data.unaccept_reason.isin(['junior synonym', 'junior homonym', 'unnecessary replacement'])) & (taxonomic_units_data.unaccept_reason.notna())]

In [95]:
itis_to_tpl_col_names = {"complete_name": "Specie", "taxon_author": "Author", "unaccept_reason": "Status"}
accepted_names = accepted_names.rename(columns=itis_to_tpl_col_names)
accepted_names["Accepted_Specie"] = accepted_names["Specie"]
accepted_names["Accepted_author"] = accepted_names["Author"]
accepted_names["Status"] = accepted_names["Status"].fillna("accepted")

In [96]:
synonym_names = synonym_names.rename(columns=itis_to_tpl_col_names)
synonym_names["Accepted_Specie"] = synonym_names["Specie"]
synonym_names["Accepted_author"] = synonym_names["Author"]

unresolved_names = unresolved_names.rename(columns=itis_to_tpl_col_names)
unresolved_names["Accepted_Specie"] = unresolved_names["Specie"]
unresolved_names["Accepted_author"] = unresolved_names["Author"]

In [99]:
if not os.path.exists(itis_processed_db_path):
    con = sqlite3.connect(itis_processed_db_path)
    accepted_names[["Specie", "Author", "Accepted_Specie", "Accepted_author", "Status", "rank_name"]].to_sql(name="Accepted_Names", con=con)
    synonym_names[["Specie", "Author", "Accepted_Specie", "Accepted_author", "Status", "rank_name"]].to_sql(name="Synonym_Names", con=con)
    unresolved_names[["Specie", "Author", "Accepted_Specie", "Accepted_author", "Status", "rank_name"]].to_sql(name="Unresolved_Names", con=con)
    con.close()